In [1]:
#!pip install neptune-client torch torchvision
#!pip install simplejson
#!pip install torchinfo

In [2]:
import os
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from dataset import EEGDataset
from torch.utils.data import random_split
import neptune.new as neptune
from torchinfo import summary


../data/study_1A_mat_simple/S_01/night_1/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_1/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_01/night_2/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_2/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_01/night_3/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_3/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_01/night_4/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_4/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_02/night_1/artefact_annotations.npy
Lables for night 0 loaded
../data/study_1A_mat_simple/S_02/night_1/EEG_raw_250hz.npy
Night 0 data loaded
debug


In [3]:
# load in the dataset

#raw_data_dir = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple'
raw_data_dir = '../data'

trainingNights = 7
testNights = 2

print("Test set")
ds2 = EEGDataset(raw_data_dir,testNights, 250, skips = trainingNights)

print("Training set")
ds1 = EEGDataset(raw_data_dir,trainingNights, 250, skips = 0) #Instantiate a dataset using the directory of data, amount of night to include and amount of samples in a segment



Test set
../data/study_1A_mat_simple/S_01/night_1/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_1/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_01/night_2/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_2/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_01/night_3/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_3/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_01/night_4/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_4/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_02/night_1/artefact_annotations.npy
../data/study_1A_mat_simple/S_02/night_1/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_02/night_2/artefact_annotations.npy
../data/study_1A_mat_simple/S_02/night_2/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_02/night_3/artefact_annotations.npy
../data/study_1A_mat_simple/S_02/night_3/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_02/night_4/artefact_annotations.npy
Lables for night 0 loaded
../data/study_1A_mat_sim

In [4]:
#Calculate class imbalance
balancing_dataloader = DataLoader(ds1, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss
artefacts = 0
good_samples = 0
for batch, (X, y) in enumerate(balancing_dataloader):
    if y == 1:
        artefacts += 1
    else:
        good_samples += 1
    if batch > 100000:
        break

class_ratio = good_samples/artefacts
print("debug")


debug


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Check for cuda 
print(f'Using {device} device')

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.LayerNorm(250),
            nn.Linear(250, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid(), 
        )

    def forward(self, x):
        #x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Using cuda device


In [6]:
learning_rate = 1e-2
batch_size = 64


model = NeuralNetwork()
model.to(device)

#pos_weight: amount of positive examples compared to negative examples. Calculate as: negative_examples/positive_examples
loss = nn.BCEWithLogitsLoss(pos_weight = class_ratio*torch.ones([batch_size]).to(device)) 

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)# Reshape to 1 dimension if using binary classification, otherwise keep dimensions from model output
        pred = pred.reshape(-1)
        pred = pred.to(device)
        yFloat = y.type(torch.FloatTensor).to(device)
        
        loss = loss_fn(pred, yFloat)
        


        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Neptune logging
        run["training/batch/loss"].log(loss)
        

        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            print("Predicted values: \n")
            print(pred)
            print("Actual values: \n")
            print(yFloat)



def test_loop(dataloader_test, model, loss_fn, test_set = True):
    size = len(dataloader_test.dataset)
    num_batches = len(dataloader_test)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader_test:
            X = X.to(device)
            y = y.to(device)
            pred = model(X).reshape(-1).to(device) # Reshape to 1 dimension if using binary classification, otherwise keep dimensions from model output
            test_loss += loss_fn(pred, y.type(torch.FloatTensor).to(device)).item()
            correct += (pred.round() == y).type(torch.float).sum().item()
            

    test_loss /= num_batches
    correct /= size
    
    
    
    if test_set:
        print(f"Test set Error: \n Test Set Accuracy: {(100*correct):>0.5f}%, Avg Test Set loss: {test_loss:>8f} \n")
        
        # Neptune logging
        run["testing/batch/test_loss"].log(test_loss)
        run["testing/batch/test_Acc"].log(100*correct)
    
    else:
        print(f"Training Set Error: \n Training Set Accuracy: {(100*correct):>0.5f}%, Avg Training Set loss: {test_loss:>8f} \n")
        
        # Neptune logging
        run["testing/batch/training_loss"].log(test_loss)
        run["testing/batch/training_Acc"].log(100*correct)

In [8]:
# Split data into train and test data (Commented out: Changed to loading different dataset class instances)
#trainSamples = int(ds1.__len__()*0.7)
#testSamples = int(ds1.__len__() - trainSamples)
#training_data, test_data = random_split(ds1, (trainSamples,testSamples), generator=torch.Generator().manual_seed(42))

#train_dataloader = DataLoader(training_data, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss
#test_dataloader = DataLoader(test_data, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss


train_dataloader = DataLoader(ds1, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss
test_dataloader = DataLoader(ds2, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss



In [9]:
# Initialize neptune
run = neptune.init(
    project="aksel.s.madsen/artefact-detection",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxYTA4NzcxMy1lYmQ2LTQ3NTctYjRhNC02Mzk1NjdjMWM0NmYifQ==",
)  # Credentials


run['config/dataset/size'] = trainingNights # dict() object
run['config/model'] = type(model).__name__
run['config/modelSummary'] = summary(model, input_size=(batch_size, 250))
run['config/optimizer'] = type(optimizer).__name__
run['config/batch_size'] = batch_size
run['config/test_night'] = testNights
run['config/learning_rate'] = learning_rate



https://app.neptune.ai/aksel.s.madsen/artefact-detection/e/AR-21
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [10]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    
    train_loop(train_dataloader, model, loss, optimizer)    
    test_loop(test_dataloader, model, loss)
    test_loop(train_dataloader, model, loss, test_set = False)

print("Done!")

Epoch 1
-------------------------------
loss: 0.718562  [    0/4951495]
Predicted values: 

tensor([0.4925, 0.4796, 0.4864, 0.4840, 0.4907, 0.4814, 0.4913, 0.4827, 0.4769,
        0.4903, 0.4865, 0.4854, 0.4948, 0.4928, 0.4905, 0.4851, 0.4767, 0.4839,
        0.4948, 0.4851, 0.4877, 0.4915, 0.4814, 0.4950, 0.4849, 0.4872, 0.4788,
        0.5009, 0.4877, 0.4893, 0.4896, 0.4893, 0.4934, 0.4854, 0.4847, 0.4827,
        0.4888, 0.4881, 0.4808, 0.4831, 0.4884, 0.4875, 0.4873, 0.4836, 0.4845,
        0.4889, 0.4859, 0.4771, 0.4820, 0.4872, 0.4800, 0.4903, 0.4897, 0.4762,
        0.4887, 0.4778, 0.4838, 0.4907, 0.4803, 0.4877, 0.4929, 0.4948, 0.4889,
        0.4807], device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
Actual values: 

tensor([1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 1., 0., 

loss: 0.515127  [576000/4951495]
Predicted values: 

tensor([0.9687, 0.0454, 0.9788, 0.0454, 0.9938, 0.0454, 0.9794, 0.0454, 0.9718,
        0.0454, 0.9819, 0.0454, 0.9885, 0.0454, 0.9718, 0.0454, 0.9729, 0.0454,
        0.9608, 0.0454, 0.9861, 0.0454, 0.9954, 0.0454, 0.9663, 0.0454, 0.9701,
        0.0454, 0.9697, 0.0454, 0.9791, 0.0454, 0.9910, 0.0454, 0.9873, 0.0454,
        0.9099, 0.0454, 0.9808, 0.0454, 0.9938, 0.0454, 0.9820, 0.0454, 0.9762,
        0.0454, 0.9911, 0.0454, 0.9661, 0.0454, 0.9946, 0.0454, 0.9932, 0.0454,
        0.9506, 0.0454, 0.9824, 0.0454, 0.9872, 0.0454, 0.9849, 0.0454, 0.9854,
        0.0454], device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
Actual values: 

tensor([1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 1., 0., 1., 0., 1., 0., 1., 0.], device='cuda:0

loss: 0.537425  [1152000/4951495]
Predicted values: 

tensor([5.1172e-01, 8.9045e-04, 9.2992e-01, 1.0305e-03, 9.1370e-01, 1.5876e-03,
        8.4248e-01, 1.6564e-03, 7.7902e-01, 1.7813e-03, 7.4443e-01, 1.5955e-03,
        8.6033e-01, 1.6469e-03, 8.2959e-01, 1.5470e-03, 9.1384e-01, 2.2429e-03,
        7.4449e-01, 1.9466e-03, 6.3230e-01, 1.7332e-03, 9.0101e-01, 1.7742e-03,
        5.5719e-01, 1.6535e-03, 8.0776e-01, 1.6107e-03, 8.7377e-01, 1.8482e-03,
        8.6129e-01, 2.6349e-03, 6.4305e-01, 1.2021e-03, 7.7964e-01, 1.0684e-03,
        8.8879e-01, 1.0775e-03, 7.5831e-01, 1.2931e-03, 8.8702e-01, 1.2474e-03,
        8.8787e-01, 1.4188e-03, 8.5572e-01, 1.9561e-03, 7.6223e-01, 1.5647e-03,
        8.3551e-01, 1.3705e-03, 4.7104e-01, 1.7123e-03, 7.4171e-01, 3.5363e-01,
        9.1571e-01, 8.5015e-02, 7.8339e-01, 1.4112e-01, 9.3036e-01, 3.1621e-02,
        7.8944e-01, 5.2742e-02, 6.2297e-01, 3.1941e-03], device='cuda:0',
       grad_fn=<ReshapeAliasBackward0>)
Actual values: 

tensor([1., 0.,

loss: 0.688379  [1664000/4951495]
Predicted values: 

tensor([3.8507e-02, 2.2057e-02, 9.2500e-02, 2.7224e-02, 2.7021e-02, 4.4916e-03,
        2.8565e-02, 1.6619e-03, 1.3377e-06, 2.3540e-02, 4.7658e-02, 1.0478e-02,
        8.1798e-03, 2.0883e-01, 3.3413e-03, 1.7649e-02, 3.4741e-02, 1.5360e-02,
        1.6734e-03, 3.6133e-03, 4.2597e-03, 4.8787e-02, 8.4526e-04, 9.2086e-04,
        2.7945e-02, 4.4585e-03, 3.8421e-02, 2.5129e-02, 1.0266e-02, 2.6655e-03,
        9.1289e-04, 8.4072e-03, 5.0458e-02, 1.4303e-03, 1.0065e-02, 4.2291e-02,
        4.0646e-04, 8.7804e-03, 1.6987e-02, 3.0098e-02, 1.1764e-03, 3.8914e-03,
        8.1898e-04, 1.5473e-03, 2.8435e-02, 2.4586e-02, 2.1748e-03, 2.0756e-02,
        1.7755e-03, 3.4623e-03, 5.9251e-03, 2.7315e-02, 7.8504e-02, 2.1663e-02,
        1.0592e-01, 4.5159e-02, 1.2854e-02, 1.6733e-02, 4.5360e-04, 4.9023e-02,
        8.3546e-04, 2.2187e-03, 2.0425e-02, 4.2574e-02], device='cuda:0',
       grad_fn=<ReshapeAliasBackward0>)
Actual values: 

tensor([1., 0.,

loss: 0.519459  [2112000/4951495]
Predicted values: 

tensor([9.9967e-01, 1.0410e-03, 9.9963e-01, 1.0410e-03, 4.5625e-04, 1.0410e-03,
        9.9929e-01, 1.0410e-03, 9.9850e-01, 1.0410e-03, 9.9745e-01, 1.0410e-03,
        9.9940e-01, 1.0410e-03, 9.9941e-01, 1.0410e-03, 9.9970e-01, 1.0410e-03,
        9.9825e-01, 1.0410e-03, 9.9965e-01, 1.0410e-03, 9.9617e-01, 1.0410e-03,
        9.9966e-01, 1.0410e-03, 9.9647e-01, 1.0410e-03, 9.9963e-01, 1.0410e-03,
        9.9942e-01, 1.0410e-03, 9.9591e-01, 1.0410e-03, 1.0410e-03, 1.0410e-03,
        9.9970e-01, 1.0410e-03, 9.9979e-01, 1.0410e-03, 9.9865e-01, 1.0410e-03,
        9.9945e-01, 1.0410e-03, 9.9921e-01, 1.0410e-03, 9.9882e-01, 1.0410e-03,
        9.9969e-01, 1.0410e-03, 9.9970e-01, 1.0410e-03, 9.9974e-01, 1.0410e-03,
        9.9969e-01, 1.0410e-03, 9.9919e-01, 1.0410e-03, 8.0142e-01, 1.0410e-03,
        9.9972e-01, 1.0410e-03, 6.9198e-04, 1.0410e-03], device='cuda:0',
       grad_fn=<ReshapeAliasBackward0>)
Actual values: 

tensor([1., 0.,

...

loss: 0.502227  [3136000/4951495]
Predicted values: 

tensor([1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.9978, 0.0000, 1.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.9973, 0.0000, 1.0000, 0.0000,
        1.0000, 0.0000, 1.0000, 0.0000, 0.9999, 0.0000, 1.0000, 0.0000, 1.0000,
        0.0000, 1.0000, 0.0000, 0.8989, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000,
        1.0000, 0.0000, 0.9773, 0.0000, 1.0000, 0.0000, 0.7811, 0.0000, 1.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.9988, 0.0000, 1.0000, 0.0000,
        1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000,
        0.0000], device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
Actual values: 

tensor([1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 1., 0., 1., 0., 1., 0., 1., 0.], device='cuda:

loss: 0.325761  [3584000/4951495]
Predicted values: 

tensor([1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 9.9999e-01,
        1.0000e+00, 1.0000e+00, 2.0079e-16, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        3.7239e-05, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.6945e-30, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00], device='cuda:0',
       grad_fn=<ReshapeAliasBackward0>)
Actual values: 

tensor([1., 1.,

loss: 0.573035  [4032000/4951495]
Predicted values: 

tensor([1.0000e+00, 1.0316e-05, 0.0000e+00, 5.5165e-16, 1.0000e+00, 7.3282e-07,
        2.3069e-02, 3.8657e-07, 2.1773e-06, 7.3571e-03, 1.0000e+00, 2.0065e-09,
        9.9987e-01, 6.9889e-12, 1.0000e+00, 2.3400e-06, 0.0000e+00, 5.7182e-06,
        7.9876e-04, 1.5808e-10, 8.6757e-03, 1.3405e-05, 5.4533e-07, 4.8383e-11,
        9.9906e-01, 1.3694e-06, 1.0780e-02, 9.5625e-10, 1.0000e+00, 2.9520e-01,
        9.9998e-01, 8.8828e-07, 1.0000e+00, 1.0986e-05, 9.9999e-01, 1.0941e-03,
        7.0596e-01, 6.3835e-11, 1.0000e+00, 2.1027e-05, 9.1488e-01, 2.5895e-12,
        0.0000e+00, 3.5250e-12, 1.0000e+00, 2.4388e-04, 9.9995e-01, 2.3254e-08,
        4.7562e-07, 2.2651e-13, 9.9999e-01, 3.8875e-07, 1.0000e+00, 9.9555e-01,
        9.9986e-01, 1.4554e-06, 1.0000e+00, 9.4805e-08, 1.0000e+00, 1.1643e-07,
        1.0000e+00, 4.6113e-02, 1.0000e+00, 6.8352e-15], device='cuda:0',
       grad_fn=<ReshapeAliasBackward0>)
Actual values: 

tensor([1., 0.,

loss: 0.579088  [4480000/4951495]
Predicted values: 

tensor([8.5380e-01, 5.7229e-36, 1.0000e+00, 0.0000e+00, 6.1183e-01, 1.5484e-38,
        9.1779e-04, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
        9.9999e-01, 1.0181e-38, 1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 9.9997e-01, 7.0339e-30, 9.9955e-01, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 3.5755e-01, 0.0000e+00, 1.0000e+00, 2.0746e-31,
        4.4518e-01, 2.4331e-36, 1.0000e+00, 4.5906e-34, 1.0000e+00, 0.0000e+00,
        9.9968e-01, 4.2054e-33, 1.0000e+00, 0.0000e+00, 2.2265e-05, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00, 9.9998e-01,
        1.0000e+00, 9.9942e-01, 1.0000e+00, 9.9891e-01, 1.0000e+00, 9.9975e-01,
        1.0000e+00, 9.9948e-01, 9.9998e-01, 9.9946e-01], device='cuda:0',
       grad_fn=<ReshapeAliasBackward0>)
Actual values: 

tensor([1., 0.,

loss: 0.545775  [4928000/4951495]
Predicted values: 

tensor([1.0000e+00, 2.6991e-05, 1.0000e+00, 1.2802e-08, 1.0000e+00, 1.1165e-03,
        1.0000e+00, 8.0086e-17, 1.0000e+00, 1.2377e-10, 1.0000e+00, 1.2267e-10,
        1.0000e+00, 2.8922e-01, 1.0000e+00, 3.8454e-08, 1.0000e+00, 1.9240e-05,
        5.3234e-26, 6.7450e-03, 1.0000e+00, 1.2673e-05, 1.0000e+00, 5.2809e-03,
        1.0000e+00, 4.3635e-08, 1.0000e+00, 8.1951e-01, 1.6497e-16, 8.2581e-04,
        1.0000e+00, 9.2989e-09, 1.0000e+00, 1.8050e-11, 1.0000e+00, 6.4088e-11,
        1.0000e+00, 8.6759e-01, 9.9999e-01, 1.8322e-07, 1.0000e+00, 6.2433e-14,
        1.0000e+00, 2.0889e-04, 1.0000e+00, 2.3620e-07, 1.0000e+00, 3.0951e-09,
        2.0653e-05, 1.1939e-07, 1.0000e+00, 4.2159e-15, 1.0000e+00, 9.5179e-01,
        1.0000e+00, 8.4738e-14, 1.0000e+00, 4.4898e-19, 1.0000e+00, 1.2695e-04,
        1.0000e+00, 2.0585e-06, 1.0000e+00, 1.4414e-15], device='cuda:0',
       grad_fn=<ReshapeAliasBackward0>)
Actual values: 

tensor([1., 0.,

In [11]:
#run.stop() # Stop the neptune logging run

In [12]:
# Test the model, and inspect the errors
from datetime import datetime
now = datetime.now()

print(f"../trained_models/model_{now.strftime('%m_%d_%Y_%H_%M_%S')}")

model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save(f"../trained_models/model_{now.strftime('%m_%d_%Y_%H_%M_%S')}") # Save

print('debug')
#randChannel = 
#testData = ds2
#self.labels[channel, start : start + self.sectionLength]


../trained_models/model_03_27_2022_15_32_32
debug


In [13]:
new_model = torch.load("../trained_models/model_03_21_2022_23_19_06")